In [1]:
import tensorflow as tf
import keras 
import pandas as pd
import numpy as np
import pyedflib
import re
import random
import math
import sklearn
import sklearn.multioutput
import sklearn.ensemble
import sklearn.neighbors
from sklearn.preprocessing import normalize
from sklearn import linear_model
from sklearn.utils import shuffle
from sklearn.model_selection import cross_val_score

Using TensorFlow backend.


In [ ]:
def makecsv(participant,dates,refresh):
    #values=[]
    for session in range(1,4):
        curr=classes[:,session-1]

        #Read self assesments (valence-arousal)
        file=open('Dataset_1/Common/Part'+participant+'SES'+str(session)+'.log','r')
        score=re.split('\n\n|\n',file.read())
        i=0
        scores=[]
        for s in score:
            i=i+1
            if i%5==0:
                scores.append(list(map(float,(s.split(' '))[-2:])))
        scores=np.asarray(scores)

        #Read markers
        file=open('Dataset_1/EEG/Part'+participant+'_IAPS_SES'+str(session)+'_EEG_fNIRS_0'+dates[session-1]+'082006.bdf.mrk','r')
        tags=file.read().split('\t')
        tag=[]
        i=0
        for t in tags[:-3]:
            if t[0].isdigit():
                i=i+1
                if i%4==0:
                    tag.append(int(t))

        marks=(sorted(set(tag)))

        #Read CSV
        df=pd.read_csv('Dataset_1/EEG/CSV/Part'+participant+'_IAPS_SES'+str(session)+'_EEG_fNIRS_0'+dates[session-1]+'082006.csv')
        
        #Take only these channels since Emotiv uses only these
        df=df[['AF3', 'FT7','F3', 'FC5', 'T7', 'P7', 'O1', 'O2', 'P8', 'T8', 'FC6', 'F4', 'FT8', 'AF4']]

        #Get FFTs jointly of the first 3 seconds of each trigger. We do this every 2 seconds
        rate=refresh[session-1]
        i=marks[0]
        for m in marks:
            i=m
            ind=marks.index(m)
            for s in range (2):
                e=i+refresh[session-1]
                d1=df[i:e+2*refresh[session-1]]
                arr=np.zeros((14,75))
                j=0
                for col in list(d1):
                    fft=(np.abs(np.fft.hfft(d1[col]))).tolist()
                    
                    fft=np.asarray(fft[1:76]) #1-25 hz
                    arr[j]=fft
                    j=j+1


                #Flatten array and join them to values
                vals=arr.flatten()
                vals=vals.tolist()
                vals.append(scores[ind][0])
                vals.append(scores[ind][1])

                values.append(np.asarray(vals).flatten())
                i=e

#Read Classes of images
file=open('Dataset_1/Common/IAPS_Classes_EEG_fNIRS.txt','r')
classes=re.split('\t|\n',file.read())
new=[(classes[:-1])[i:i+3] for i in range(0, len(classes)-1, 3)]
classes=np.asarray(new)


values=[]
#Participants
#1
dates=['3','7','8']
refresh=[256,1024,1024]
makecsv('1',dates,refresh)

#2
dates=['7','8','9']
refresh=[1024,1024,1024]
makecsv('2',dates,refresh)

#3
dates=['7','8','8']
makecsv('3',dates,refresh)


#4
dates=['7','8','9']
makecsv('4',dates,refresh)


#5
dates=['8','9','9']
makecsv('5',dates,refresh)

np.savetxt("Dataset1_3SecJoinedFT_Flat.csv",np.asarray(values),delimiter=",",fmt='%s')

In [ ]:
#DATASET 2 FORMATTING
values=[]
for part in range(1,24):
    for vid in range (1,19):
        df=pd.read_csv('Dataset_2/MatlabOutput/Part'+str(part)+'Vid'+str(vid)+'.csv',header=None).values
        score=pd.read_csv('Dataset_2/MatlabOutput/Part'+str(part)+'Vid'+str(vid)+'Score.csv',header=None).values[0]
        
        
        #Removing DC Offset
        #for x in range (df.shape[1]):
        #    m=np.mean(df[:,x])
        #    df[:,x]=df[:,x]-m
        #df=df-4200
        
        
        
        start=500
        #Get FFTs separately of the first 3 seconds of each trigger. We do this every 2 seconds
        for x in range (10):
            end=start+128
            d1=df[start:end,:]
            d2=df[end:end+128,:]
            d3=df[end+128:end+256,:]
            arr=np.zeros((14,75))
            j=0
            for col in range(14):
                fft1=(np.abs(np.fft.hfft(d1[:,col]))).tolist()
                fft2=(np.abs(np.fft.hfft(d2[:,col]))).tolist()
                fft3=(np.abs(np.fft.hfft(d3[:,col]))).tolist()
                fft=np.asarray(fft1[1:26]+fft2[1:26]+fft3[1:26]) #1-25 hz
                
                arr[j]=fft
                j=j+1
                

            #Flatten and join
            vals=arr.flatten()
            #vals=vals/np.sum(vals)
            vals=vals.tolist()
            vals.append(score[0])
            vals.append(score[1])
            #vals.append(curr[ind])
            values.append(np.asarray(vals).flatten())
            start=end    
np.savetxt("Dataset2_3SecSeparateFT_FlatWithOS.csv",np.asarray(values),delimiter=",",fmt='%s')

The next cells below are just a thorough test of the datasets with different types of classifiers. Doing this, I chose the best formatting for the datasets and the best classifiers.

In [ ]:


#Training classifier

#Reading dataset file
dataset=pd.read_csv('Dataset2_3SecJoinedFT_FlatOS.csv',header=None).values


x=dataset[:,:-2]
#x=x*1000
y=dataset[:,-2:]

print(x.shape)
x,y=shuffle(x,y)
train=x[:3000]
test=x[3000:]

y1=y[:,0].astype(int)
y2=y[:,1].astype(int)
y1train=y1[:3000].astype(int)
y1test=y1[3000:].astype(int)
y2train=y2[:3000].astype(int)
y2test=y2[3000:].astype(int)

In [ ]:
#BEST

from sklearn.neural_network import MLPClassifier

#Reading dataset file
dataset=pd.read_csv('Dataset2_3SecSeparateFT_FlatWithOS.csv',header=None).values


x=dataset[:,:-2]
x=x*5000
y=dataset[:,-2:]

print(x.shape)
x,y=shuffle(x,y)
train=x[:500]
test=x[500:]

y1=y[:,0].astype(int)
y2=y[:,1].astype(int)
y1train=y1[:500].astype(int)
y1test=y1[500:].astype(int)
y2train=y2[:500].astype(int)
y2test=y2[500:].astype(int)
print ('3 Sec Separate')
mlr1=MLPClassifier(solver='adam',activation='relu',alpha=1e-5,hidden_layer_sizes=(400,),max_iter=1000, random_state=0)
mlr2=MLPClassifier(solver='adam',activation='relu',alpha=1e-5,hidden_layer_sizes=(400,),max_iter=1000, random_state=0)
print(np.mean(cross_val_score(mlr1,x,y1,cv=3)))
print(np.mean(cross_val_score(mlr2,x,y2,cv=3)))
mlr1.fit(train,y1train)
mlr2.fit(train,y2train)
error1=0
error2=0
for x in range(300):
    #i=random.randint(0,300)
    p1=min(max(mlr1.predict(test[x].reshape(1,-1))[0],1),10)
    p2=min(max(mlr2.predict(test[x].reshape(1,-1))[0],1),10)
    if p1==y1test[x]:
        error1=error1+3
    if (p1-y1test[x])**2==1:
        error1=error1+1
    if p2==y2test[x]:
        error2=error2+3
    if (p2-y2test[x])**2==1:
        error2=error2+1
    #error1=error1+(y1test[x]-p1)**2
    #error2=error2+(y2test[x]-p2)**2
    if x%10==0:
        print(y1test[x],y2test[x])
        print(p1,p2)
        print()
print('Valence:',error1)
print('Arousal:',error2)




#Reading dataset file
dataset=pd.read_csv('Dataset2_3SecJoinedFT_Flat.csv',header=None).values


x=dataset[:,:-2]
x=x*5000
y=dataset[:,-2:]

print(x.shape)
x,y=shuffle(x,y)
train=x[:500]
test=x[500:]

y1=y[:,0].astype(int)
y2=y[:,1].astype(int)
y1train=y1[:500].astype(int)
y1test=y1[500:].astype(int)
y2train=y2[:500].astype(int)
y2test=y2[500:].astype(int)
print ('\n\n3 Sec Joined')
mlr1=MLPClassifier(solver='adam',activation='relu',alpha=1e-5,hidden_layer_sizes=(400,),max_iter=1000, random_state=0)
mlr2=MLPClassifier(solver='adam',activation='relu',alpha=1e-5,hidden_layer_sizes=(400,),max_iter=1000, random_state=0)
print(np.mean(cross_val_score(mlr1,x,y1,cv=3)))
print(np.mean(cross_val_score(mlr2,x,y2,cv=3)))
mlr1.fit(train,y1train)
mlr2.fit(train,y2train)
error1=0
error2=0
for x in range(300):
    #i=random.randint(0,300)
    p1=min(max(mlr1.predict(test[x].reshape(1,-1))[0],1),10)
    p2=min(max(mlr2.predict(test[x].reshape(1,-1))[0],1),10)
    if p1==y1test[x]:
        error1=error1+3
    if (p1-y1test[x])**2==1:
        error1=error1+1
    if p2==y2test[x]:
        error2=error2+3
    if (p2-y2test[x])**2==1:
        error2=error2+1
    #error1=error1+(y1test[x]-p1)**2
    #error2=error2+(y2test[x]-p2)**2
    if x%10==0:
        print(y1test[x],y2test[x])
        print(p1,p2)
        print()
print('Valence:',error1)
print('Arousal:',error2)

In [ ]:
clf=sklearn.multioutput.MultiOutputClassifier(sklearn.svm.SVC(gamma='auto'), n_jobs=2)
np.mean(cross_val_score(clf,x,y,cv=10))

In [ ]:
svm1=sklearn.svm.SVC(gamma='auto')
print(np.mean(cross_val_score(svm1,x,y1,cv=10)))
print(np.mean(cross_val_score(svm1,x,y2,cv=10)))

In [ ]:
rf=sklearn.ensemble.RandomForestClassifier(n_estimators=50,criterion='entropy',max_depth=400)
print(np.mean(cross_val_score(rf,x,y1,cv=10)))
print(np.mean(cross_val_score(rf,x,y2,cv=10)))

In [ ]:
lr=linear_model.LogisticRegression(solver='saga',max_iter=4000,multi_class='multinomial')
print(np.mean(cross_val_score(lr,x,y1,cv=10)))
print(np.mean(cross_val_score(lr,x,y2,cv=10)))

In [ ]:
knn=sklearn.neighbors.KNeighborsClassifier(n_neighbors=5,algorithm='auto')
print(np.mean(cross_val_score(knn,x,y1,cv=10)))
print(np.mean(cross_val_score(knn,x,y2,cv=10)))

In [ ]:
from sklearn.neural_network import MLPClassifier
mlp1=MLPClassifier(solver='adam',activation='relu', alpha=1e-3,hidden_layer_sizes=(600,),max_iter=1000, random_state=0)
mlp2=MLPClassifier(solver='adam',activation='relu', alpha=1e-3,hidden_layer_sizes=(600,),max_iter=1000, random_state=0)
print(np.mean(cross_val_score(mlp1,x,y1,cv=10)))
print(np.mean(cross_val_score(mlp2,x,y2,cv=10)))

In [ ]:
svr1=sklearn.svm.SVR(kernel='poly',gamma='scale')
svr2=sklearn.svm.SVR(kernel='poly',gamma='scale')
svr1.fit(train,y1train)
svr2.fit(train,y2train)
error1=0
error2=0
for x in range(300):
    i=random.randint(0,300)
    p1=min(max(svr1.predict(test[x].reshape(1,-1))[0],1),10)
    p2=min(max(svr2.predict(test[x].reshape(1,-1))[0],1),10)
    if round(p1)==y1test[x]:
        error1=error1+3
    if (round(p1)-y1test[x])**2==1:
        error1=error1+1
    if round(p2)==y2test[x]:
        error2=error2+3
    if (round(p2)-y2test[x])**2==1:
        error2=error2+1
    #error1=error1+(y1test[x]-p1)**2
    #error2=error2+(y2test[x]-p2)**2
    if x%10==0:
        print(y1test[x],y2test[x])
        print(p1,p2)
        print()
print('Valence:',error1)
print('Arousal:',error2)

In [ ]:
rid1=sklearn.linear_model.Ridge(alpha=1.0)
rid2=sklearn.linear_model.Ridge(alpha=1.0)
rid1.fit(train,y1train)
rid2.fit(train,y2train)
error1=0
error2=0
for x in range(300):
    i=random.randint(0,300)
    p1=min(max(rid1.predict(test[x].reshape(1,-1))[0],1),10)
    p2=min(max(rid2.predict(test[x].reshape(1,-1))[0],1),10)
    if round(p1)==y1test[x]:
        error1=error1+3
    if (round(p1)-y1test[x])**2==1:
        error1=error1+1
    if round(p2)==y2test[x]:
        error2=error2+3
    if (round(p2)-y2test[x])**2==1:
        error2=error2+1
    #error1=error1+(y1test[x]-p1)**2
    #error2=error2+(y2test[x]-p2)**2
    if x%10==0:
        print(y1test[x],y2test[x])
        print(p1,p2)
        print()
print('Valence:',error1)
print('Arousal:',error2)

In [ ]:
lars1=sklearn.linear_model.LassoLars(alpha=1.0)
lars2=sklearn.linear_model.LassoLars(alpha=1.0)
lars1.fit(train,y1train)
lars2.fit(train,y2train)
error1=0
error2=0
for x in range(300):
    i=random.randint(0,300)
    p1=min(max(lars1.predict(test[x].reshape(1,-1))[0],1),10)
    p2=min(max(lars2.predict(test[x].reshape(1,-1))[0],1),10)
    if round(p1)==y1test[x]:
        error1=error1+3
    if (round(p1)-y1test[x])**2==1:
        error1=error1+1
    if round(p2)==y2test[x]:
        error2=error2+3
    if (round(p2)-y2test[x])**2==1:
        error2=error2+1
    #error1=error1+(y1test[x]-p1)**2
    #error2=error2+(y2test[x]-p2)**2
    if x%10==0:
        print(y1test[x],y2test[x])
        print(p1,p2)
        print()
print('Valence:',error1)
print('Arousal:',error2)

In [ ]:
bay1=sklearn.linear_model.BayesianRidge(n_iter=1000)
bay2=sklearn.linear_model.BayesianRidge(n_iter=1000)
bay1.fit(train,y1train)
bay2.fit(train,y2train)
error1=0
error2=0
for x in range(300):
    i=random.randint(0,300)
    p1=min(max(bay1.predict(test[x].reshape(1,-1))[0],1),10)
    p2=min(max(bay2.predict(test[x].reshape(1,-1))[0],1),10)
    if round(p1)==y1test[x]:
        error1=error1+3
    if (round(p1)-y1test[x])**2==1:
        error1=error1+1
    if round(p2)==y2test[x]:
        error2=error2+3
    if (round(p2)-y2test[x])**2==1:
        error2=error2+1
    #error1=error1+(y1test[x]-p1)**2
    #error2=error2+(y2test[x]-p2)**2
    if x%10==0:
        print(y1test[x],y2test[x])
        print(p1,p2)
        print()
print('Valence:',error1)
print('Arousal:',error2)

In [ ]:
svr1=sklearn.svm.SVR(kernel='sigmoid',gamma='scale')
svr2=sklearn.svm.SVR(kernel='sigmoid',gamma='scale')
svr1.fit(train,y1train)
svr2.fit(train,y2train)
error1=0
error2=0
for x in range(300):
    i=random.randint(0,300)
    p1=min(max(svr1.predict(test[x].reshape(1,-1))[0],1),10)
    p2=min(max(svr2.predict(test[x].reshape(1,-1))[0],1),10)
    if round(p1)==y1test[x]:
        error1=error1+3
    if (round(p1)-y1test[x])**2==1:
        error1=error1+1
    if round(p2)==y2test[x]:
        error2=error2+3
    if (round(p2)-y2test[x])**2==1:
        error2=error2+1
    #error1=error1+(y1test[x]-p1)**2
    #error2=error2+(y2test[x]-p2)**2
    #error1=error1+(y1test[x]-p1)**2
    #error2=error2+(y2test[x]-p2)**2
    if x%10==0:
        print(y1test[x],y2test[x])
        print(p1,p2)
        print()
print('Valence:',error1)
print('Arousal:',error2)

In [ ]:
per1=sklearn.linear_model.Perceptron(max_iter=2000)
per2=sklearn.linear_model.Perceptron(max_iter=2000)
per1.fit(train,y1train)
per2.fit(train,y2train)
error1=0
error2=0
for x in range(300):
    i=random.randint(0,300)
    p1=min(max(per1.predict(test[x].reshape(1,-1))[0],1),10)
    p2=min(max(per2.predict(test[x].reshape(1,-1))[0],1),10)
    if p1==y1test[x]:
        error1=error1+3
    if (p1-y1test[x])**2==1:
        error1=error1+1
    if p2==y2test[x]:
        error2=error2+3
    if (p2-y2test[x])**2==1:
        error2=error2+1
    #error1=error1+(y1test[x]-p1)**2
    #error2=error2+(y2test[x]-p2)**2
    if x%10==0:
        print(y1test[x],y2test[x])
        print(p1,p2)
        print()
print('Valence:',error1)
print('Arousal:',error2)

In [ ]:
the1=sklearn.linear_model.TheilSenRegressor()
the2=sklearn.linear_model.TheilSenRegressor()
the1.fit(train,y1train)
the2.fit(train,y2train)
error1=0
error2=0
for x in range(300):
    i=random.randint(0,300)
    p1=min(max(the1.predict(test[x].reshape(1,-1))[0],1),10)
    p2=min(max(the2.predict(test[x].reshape(1,-1))[0],1),10)
    if round(p1)==y1test[x]:
        error1=error1+3
    if (round(p1)-y1test[x])**2==1:
        error1=error1+1
    if round(p2)==y2test[x]:
        error2=error2+3
    if (round(p2)-y2test[x])**2==1:
        error2=error2+1
    #error1=error1+(y1test[x]-p1)**2
    #error2=error2+(y2test[x]-p2)**2
    #error1=error1+(y1test[x]-p1)**2
    #error2=error2+(y2test[x]-p2)**2
    if x%10==0:
        print(y1test[x],y2test[x])
        print(p1,p2)
        print()
print('Valence:',error1)
print('Arousal:',error2)

In [ ]:
#Separate further class dimension, try more regressors
from sklearn.linear_model import PassiveAggressiveRegressor

pas1=PassiveAggressiveRegressor()
pas2=PassiveAggressiveRegressor()
pas1.fit(train,y1train)
pas2.fit(train,y2train)
error1=0
error2=0
for x in range(300):
    i=random.randint(0,300)
    p1=min(max(pas1.predict(test[x].reshape(1,-1))[0],1),10)
    p2=min(max(pas2.predict(test[x].reshape(1,-1))[0],1),10)
    if round(p1)==y1test[x]:
        error1=error1+3
    if (round(p1)-y1test[x])**2==1:
        error1=error1+1
    if round(p2)==y2test[x]:
        error2=error2+3
    if (round(p2)-y2test[x])**2==1:
        error2=error2+1
    #error1=error1+(y1test[x]-p1)**2
    #error2=error2+(y2test[x]-p2)**2
    #error1=error1+(y1test[x]-p1)**2
    #error2=error2+(y2test[x]-p2)**2
    if x%10==0:
        print(y1test[x],y2test[x])
        print(p1,p2)
        print()
print('Valence:',error1)
print('Arousal:',error2)

In [ ]:
from sklearn.linear_model import SGDRegressor

sgd1=SGDRegressor(max_iter=5000,tol=0.01)
sgd2=SGDRegressor(max_iter=5000,tol=0.01)
sgd1.fit(train,y1train)
sgd2.fit(train,y2train)
error1=0
error2=0
for x in range(300):
    #i=random.randint(0,300)
    p1=min(max(sgd1.predict(test[x].reshape(1,-1))[0],1),10)
    p2=min(max(sgd2.predict(test[x].reshape(1,-1))[0],1),10)
    if round(p1)==y1test[x]:
        error1=error1+3
    if (round(p1)-y1test[x])**2==1:
        error1=error1+1
    if round(p2)==y2test[x]:
        error2=error2+3
    if (round(p2)-y2test[x])**2==1:
        error2=error2+1
    #error1=error1+(y1test[x]-p1)**2
    #error2=error2+(y2test[x]-p2)**2
    #error1=error1+(y1test[x]-p1)**2
    #error2=error2+(y2test[x]-p2)**2
    if x%10==0:
        print(y1test[x],y2test[x])
        print(p1,p2)
        print()
print('Valence:',error1)
print('Arousal:',error2)

In [ ]:
from sklearn.linear_model import HuberRegressor

hub1=HuberRegressor(max_iter=1000)
hub2=HuberRegressor(max_iter=1000)
hub1.fit(train,y1train)
hub2.fit(train,y2train)
error1=0
error2=0
for x in range(300):
    #i=random.randint(0,300)
    p1=min(max(hub1.predict(test[x].reshape(1,-1))[0],1),10)
    p2=min(max(hub2.predict(test[x].reshape(1,-1))[0],1),10)
    if round(p1)==y1test[x]:
        error1=error1+3
    if (round(p1)-y1test[x])**2==1:
        error1=error1+1
    if round(p2)==y2test[x]:
        error2=error2+3
    if (round(p2)-y2test[x])**2==1:
        error2=error2+1
    #error1=error1+(y1test[x]-p1)**2
    #error2=error2+(y2test[x]-p2)**2
    #error1=error1+(y1test[x]-p1)**2
    #error2=error2+(y2test[x]-p2)**2
    if x%10==0:
        print(y1test[x],y2test[x])
        print(p1,p2)
        print()
print('Valence:',error1)
print('Arousal:',error2)

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

knn1=KNeighborsRegressor(n_neighbors=5)
knn2=KNeighborsRegressor(n_neighbors=5)
knn1.fit(train,y1train)
knn2.fit(train,y2train)
error1=0
error2=0
for x in range(300):
    #i=random.randint(0,300)
    p1=min(max(knn1.predict(test[x].reshape(1,-1))[0],1),10)
    p2=min(max(knn2.predict(test[x].reshape(1,-1))[0],1),10)
    if round(p1)==y1test[x]:
        error1=error1+3
    if (round(p1)-y1test[x])**2==1:
        error1=error1+1
    if round(p2)==y2test[x]:
        error2=error2+3
    if (round(p2)-y2test[x])**2==1:
        error2=error2+1
    #error1=error1+(y1test[x]-p1)**2
    #error2=error2+(y2test[x]-p2)**2
    if x%10==0:
        print(y1test[x],y2test[x])
        print(p1,p2)
        print()
print('Valence:',error1)
print('Arousal:',error2)

In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel

kernel = DotProduct() + WhiteKernel()
gpr1=GaussianProcessRegressor(kernel=kernel, random_state=0)
gpr2=GaussianProcessRegressor(kernel=kernel,random_state=0)
gpr1.fit(train,y1train)
gpr2.fit(train,y2train)
error1=0
error2=0
for x in range(300):
    #i=random.randint(0,300)
    p1=min(max(gpr1.predict(test[x].reshape(1,-1))[0],1),10)
    p2=min(max(gpr2.predict(test[x].reshape(1,-1))[0],1),10)
    if round(p1)==y1test[x]:
        error1=error1+3
    if (round(p1)-y1test[x])**2==1:
        error1=error1+1
    if round(p2)==y2test[x]:
        error2=error2+3
    if (round(p2)-y2test[x])**2==1:
        error2=error2+1
    #error1=error1+(y1test[x]-p1)**2
    #error2=error2+(y2test[x]-p2)**2
    if x%10==0:
        print(y1test[x],y2test[x])
        print(p1,p2)
        print()
print('Valence:',error1)
print('Arousal:',error2)

In [ ]:
from sklearn import tree

dec1=tree.DecisionTreeRegressor()
dec2=tree.DecisionTreeRegressor()
dec1.fit(train,y1train)
dec2.fit(train,y2train)
error1=0
error2=0
for x in range(300):
    #i=random.randint(0,300)
    p1=min(max(dec1.predict(test[x].reshape(1,-1))[0],1),10)
    p2=min(max(dec2.predict(test[x].reshape(1,-1))[0],1),10)
    if round(p1)==y1test[x]:
        error1=error1+3
    if (round(p1)-y1test[x])**2==1:
        error1=error1+1
    if round(p2)==y2test[x]:
        error2=error2+3
    if (round(p2)-y2test[x])**2==1:
        error2=error2+1
    #error1=error1+(y1test[x]-p1)**2
    #error2=error2+(y2test[x]-p2)**2
    if x%10==0:
        print(y1test[x],y2test[x])
        print(p1,p2)
        print()
print('Valence:',error1)
print('Arousal:',error2)

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

dec1=GradientBoostingRegressor(n_estimators=100, learning_rate=0.1,max_depth=200, random_state=0, loss='ls')
dec2=GradientBoostingRegressor(n_estimators=100, learning_rate=0.1,max_depth=200, random_state=0, loss='ls')
dec1.fit(train,y1train)
dec2.fit(train,y2train)
error1=0
error2=0
for x in range(300):
    #i=random.randint(0,300)
    p1=min(max(dec1.predict(test[x].reshape(1,-1))[0],1),10)
    p2=min(max(dec2.predict(test[x].reshape(1,-1))[0],1),10)
    if round(p1)==y1test[x]:
        error1=error1+3
    if (round(p1)-y1test[x])**2==1:
        error1=error1+1
    if round(p2)==y2test[x]:
        error2=error2+3
    if (round(p2)-y2test[x])**2==1:
        error2=error2+1
    #error1=error1+(y1test[x]-p1)**2
    #error2=error2+(y2test[x]-p2)**2
    if x%10==0:
        print(y1test[x],y2test[x])
        print(p1,p2)
        print()
print('Valence:',error1)
print('Arousal:',error2)

In [ ]:
from sklearn.neural_network import MLPRegressor

mlr1=MLPRegressor(solver='adam',activation='relu',alpha=1e-5,hidden_layer_sizes=(400,),max_iter=1000, random_state=0)
mlr2=MLPRegressor(solver='adam',activation='relu',alpha=1e-5,hidden_layer_sizes=(400,),max_iter=1000, random_state=0)
mlr1.fit(train,y1train)
mlr2.fit(train,y2train)
error1=0
error2=0
for x in range(300):
    #i=random.randint(0,300)
    p1=min(max(mlr1.predict(test[x].reshape(1,-1))[0],1),10)
    p2=min(max(mlr2.predict(test[x].reshape(1,-1))[0],1),10)
    if round(p1)==y1test[x]:
        error1=error1+3
    if (round(p1)-y1test[x])**2==1:
        error1=error1+1
    if round(p2)==y2test[x]:
        error2=error2+3
    if (round(p2)-y2test[x])**2==1:
        error2=error2+1
    #error1=error1+(y1test[x]-p1)**2
    #error2=error2+(y2test[x]-p2)**2
    if x%10==0:
        print(y1test[x],y2test[x])
        print(p1,p2)
        print()
print('Valence:',error1)
print('Arousal:',error2)

In [ ]:
from sklearn.neural_network import MLPClassifier

mlr1=MLPClassifier(solver='adam',activation='relu',alpha=1e-6,hidden_layer_sizes=(600,),max_iter=2000, random_state=0)
mlr2=MLPClassifier(solver='adam',activation='relu',alpha=1e-6,hidden_layer_sizes=(600,),max_iter=2000, random_state=0)
mlr1.fit(train,y1train)
mlr2.fit(train,y2train)
error1=0
error2=0
for x in range(300):
    #i=random.randint(0,300)
    p1=min(max(mlr1.predict(test[x].reshape(1,-1))[0],1),10)
    p2=min(max(mlr2.predict(test[x].reshape(1,-1))[0],1),10)
    if round(p1)==y1test[x]:
        error1=error1+3
    if (round(p1)-y1test[x])**2==1:
        error1=error1+1
    if round(p2)==y2test[x]:
        error2=error2+3
    if (round(p2)-y2test[x])**2==1:
        error2=error2+1
    #error1=error1+(y1test[x]-p1)**2
    #error2=error2+(y2test[x]-p2)**2
    if x%10==0:
        print(y1test[x],y2test[x])
        print(p1,p2)
        print()
print('Valence:',error1)
print('Arousal:',error2)

In [ ]:
#Receiving data from OpenBCI GUI TEST
import argparse
import time
import asyncio
import numpy as np
import pandas as pd
from pythonosc import dispatcher
from pythonosc import osc_server
from pythonosc import udp_client
from pythonosc import osc_message_builder
from pythonosc import osc_bundle_builder
from typing import List, Any

from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPClassifier

#Reading and joining both datasets
dataset=pd.read_csv('Dataset1_3SecJoinedFT_Flat.csv',header=None).values
ds2=pd.read_csv('Dataset2_3SecJoinedFT_FlatOS.csv',header=None).values
dataset=np.append(dataset,ds2,axis=0)
x=dataset[:,:-2]

y=dataset[:,-2:]


y1=y[:,0].astype(int)
y2=y[:,1].astype(int)


#Train classifiers (MLP and KNN)
print('Training Classifiers...')
mlr1=MLPClassifier(solver='adam',activation='relu',alpha=1e-6,hidden_layer_sizes=(600,),max_iter=2000, random_state=0)
mlr2=MLPClassifier(solver='adam',activation='relu',alpha=1e-6,hidden_layer_sizes=(600,),max_iter=2000, random_state=0)
mlr1.fit(x,y1)
mlr2.fit(x,y2)

knn1=KNeighborsRegressor(n_neighbors=5)
knn2=KNeighborsRegressor(n_neighbors=5)
knn1.fit(x,y1)
knn2.fit(x,y2)

print('Done!')

#Intialize list of Timeseries values
vals=[]
def print_test(addr,*args:List[Any]):
    vals.append(np.asarray(args)[2:])

def classify(using):
    arr=np.zeros((14,75))
    for col in range(14):
        fft=(np.abs(np.fft.hfft(using[:,col]))).tolist()
        fft=np.asarray(fft[1:76]) #1-25 hz
        arr[col]=fft
    arr=arr.flatten()
    val=mlr1.predict(arr.reshape(1,-1))[0]
    aro=mlr2.predict(arr.reshape(1,-1))[0]
    print("\n\nValence: MLP",val,"\tKNN",knn1.predict(arr.reshape(1,-1))[0])
    print("Arousal: MLP",aro,"\tKNN",knn2.predict(arr.reshape(1,-1))[0])
    return val,aro
    
    
dispatcher = dispatcher.Dispatcher()
dispatcher.map("/openbci", print_test)
unity=udp_client.SimpleUDPClient("127.0.0.1", 5555)
    
async def loop():
    while True:
        curr=np.asarray(vals)
        if (curr.shape[0]>=375):
            curr=curr[-375:]
            valence,arousal=classify(curr)
            unity.send_message("/python",[float(valence),float(arousal)])
            
        await asyncio.sleep(1)


async def init_main():
    server = osc_server.AsyncIOOSCUDPServer(('127.0.0.1', 12347), dispatcher, asyncio.get_event_loop())
    transport, protocol = await server.create_serve_endpoint()  # Create datagram endpoint and start serving

    await loop()  # Enter main loop of program

    transport.close()  # Clean up serve endpoint


await init_main()

In [ ]:
valence=123
unity=udp_client.SimpleUDPClient("127.0.0.1", 5555)
unity.send_message("/python",valence)

In [ ]:
df=pd.read_csv('Dataset_1/EEG/CSV/Part1_IAPS_SES1_EEG_fNIRS_03082006.csv')
        #df=df.drop(['F5','F8','AF7','AF8','AFz','# Fp1','Fp2','Fpz','F7','F6'],axis=1)#10 electrodes are not used
        #if participant=='1' and session==1:
         #   df=df.drop(['EXG1','EXG2','EXG3','EXG4','EXG5','EXG6','EXG7','EXG8','Status'],axis=1)
        #else:
         #   df=df.drop(['GSR1','GSR2','Erg1','Erg2','Resp','Plet','Temp','Status'],axis=1)
df=df[['AF3', 'FT7','F3', 'FC5', 'T7', 'P7', 'O1', 'O2', 'P8', 'T8', 'FC6', 'F4', 'FT8', 'AF4']]
df

In [ ]:
#Old way

def makecsv(participant,dates,refresh):
    #values=[]
    for session in range(1,4):
        curr=classes[:,session-1]

        #Read self assesments (valence-arousal)
        file=open('Dataset_1/Common/Part'+participant+'SES'+str(session)+'.log','r')
        score=re.split('\n\n|\n',file.read())
        i=0
        scores=[]
        for s in score:
            i=i+1
            if i%5==0:
                scores.append(list(map(float,(s.split(' '))[-2:])))
        scores=np.asarray(scores)

        #Read markers
        file=open('Dataset_1/EEG/Part'+participant+'_IAPS_SES'+str(session)+'_EEG_fNIRS_0'+dates[session-1]+'082006.bdf.mrk','r')
        tags=file.read().split('\t')
        tag=[]
        i=0
        for t in tags[:-3]:
            if t[0].isdigit():
                i=i+1
                if i%4==0:
                    tag.append(int(t))

        marks=(sorted(set(tag)))

        #Read CSV
        df=pd.read_csv('Dataset_1/EEG/CSV/Part'+participant+'_IAPS_SES'+str(session)+'_EEG_fNIRS_0'+dates[session-1]+'082006.csv')
        df=df.drop(['F5','F8','AF7','AF8','AFz','# Fp1','Fp2','Fpz','F7','F6'],axis=1)#10 electrodes are not used
        if participant=='1' and session==1:
            df=df.drop(['EXG1','EXG2','EXG3','EXG4','EXG5','EXG6','EXG7','EXG8','Status'],axis=1)
        else:
            df=df.drop(['GSR1','GSR2','Erg1','Erg2','Resp','Plet','Temp','Status'],axis=1)


        #Get FFTS of the first 3 seconds of each trigger. We take the FFT for every 2 seconds
        rate=refresh[session-1]
        i=marks[0]
        for m in marks:
            i=m
            ind=marks.index(m)
            for s in range (2):
                e=i+refresh[session-1]
                d=df[i:e+refresh[session-1]]
                arr=np.zeros((54,50))
                j=0
                for col in list(d):
                    fft=(np.abs(np.fft.hfft(d[col]))).tolist()
                    #del fft[1::2]
                    fft=np.asarray(fft[1:51]) #1-25 hz
                    arr[j]=fft
                    j=j+1

                #Mean and normalize values
                vals=np.mean(arr,axis=0)
                #vals=vals/np.sum(vals)
                vals=vals.tolist()
                vals.append(scores[ind][0])
                vals.append(scores[ind][1])
                #vals.append(curr[ind])
                values.append(np.asarray(vals).flatten())
                i=e

    #np.savetxt("Participant"+participant+".csv",np.asarray(values),delimiter=",",fmt='%s')
    
#Read Classes of images
file=open('Dataset_1/Common/IAPS_Classes_EEG_fNIRS.txt','r')
classes=re.split('\t|\n',file.read())
new=[(classes[:-1])[i:i+3] for i in range(0, len(classes)-1, 3)]
classes=np.asarray(new)


values=[]
#Participants
#1
dates=['3','7','8']
refresh=[256,1024,1024]
makecsv('1',dates,refresh)

#2
dates=['7','8','9']
refresh=[1024,1024,1024]
makecsv('2',dates,refresh)

#3
dates=['7','8','8']
makecsv('3',dates,refresh)


#4
dates=['7','8','9']
makecsv('4',dates,refresh)


#5
dates=['8','9','9']
makecsv('5',dates,refresh)

np.savetxt("Dataset1Separate.csv",np.asarray(values),delimiter=",",fmt='%s')


#DATASET 2 FORMATTING
values=[]
for part in range(1,24):
    #values=[]
    for vid in range (1,19):
        df=pd.read_csv('Dataset_2/MatlabOutput/Part'+str(part)+'Vid'+str(vid)+'.csv',header=None).values
        score=pd.read_csv('Dataset_2/MatlabOutput/Part'+str(part)+'Vid'+str(vid)+'Score.csv',header=None).values[0]
        #Removing DC Offset
        for x in range (df.shape[1]):
            m=np.mean(df[:,x])
            df[:,x]=df[:,x]-m
        start=500
        for x in range (10):
            end=start+128
            d=df[start:(end+128),:]
            arr=np.zeros((14,75))
            for col in range(14):
                fft=(np.abs(np.fft.hfft(d[:,col]))).tolist()
                #del fft[1::2]
                fft=np.asarray(fft[1:76]) #1-25 hz
                arr[j]=fft
            print('Valence:',knn1.predict(test[x].reshape(1,-1))
            print('Arousal:',knn2.predict(test[x].reshape(1,-1))
                
            #Mean and normalize values
            vals=np.mean(arr,axis=0)
            #vals=vals/np.sum(vals)
            vals=vals.tolist()
            vals.append(score[0])
            vals.append(score[1])
            #vals.append(curr[ind])
            values.append(np.asarray(vals).flatten())
            start=end
    #np.savetxt("Participant"+str(part)+".csv",np.asarray(values),delimiter=",",fmt='%s')
    
np.savetxt("Dataset2Separate.csv",np.asarray(values),delimiter=",",fmt='%s')